In [7]:
import time
import secrets
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad, unpad

# Baudot code dictionary
baudot = {
    ' ': '00000', 't': '00001', 'o': '00011', 'h': '00101', 'n': '00110',
    'm': '00111', 'l': '01001', 'r': '01010', 'g': '01011', 'i': '01100',
    'p': '01101', 'c': '01110', 'v': '01111', 'e': '10000', 'z': '10001',
    'd': '10010', 'b': '10011', 's': '10100', 'y': '10101', 'f': '10110',
    'x': '10111', 'a': '11000', 'w': '11001', 'j': '11010', 'u': '11100',
    'q': '11101', 'k': '11110'
}
reverse_baudot = {v: k for k, v in baudot.items()}

def to_baudot_bits(text):
    return ''.join([baudot[c] for c in text.lower() if c in baudot])

def bits_to_bytes(bits):
    while len(bits) % 8 != 0:
        bits += '0'
    return [int(bits[i:i+8], 2) for i in range(0, len(bits), 8)]

def bytes_to_bits(byte_list):
    return ''.join([format(b, '08b') for b in byte_list])

def from_baudot_bits(bits):
    return ''.join([reverse_baudot.get(bits[i:i+5], '?') for i in range(0, len(bits), 5)])

def tent_map(x0, r, length):
    x = x0
    chaos = []
    for _ in range(length):
        x = r * x if x < 0.5 else r * (1 - x)
        chaos.append(int(x * 256) % 256)
    return chaos

def xor_lists(a, b):
    return [x ^ y for x, y in zip(a, b)]

# AES ENCRYPTION
def aes_encrypt(byte_list, key):
    key = SHA256.new(key).digest()
    iv = secrets.token_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    padded = pad(bytes(byte_list), 16)
    ciphertext = cipher.encrypt(padded)
    return b64encode(iv + ciphertext).decode()

def aes_decrypt(data_b64, key):
    key = SHA256.new(key).digest()
    raw = b64decode(data_b64)
    iv, ciphertext = raw[:16], raw[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted = unpad(cipher.decrypt(ciphertext), 16)
    return list(decrypted)

def encrypt(text, key, x0=0.7, r=1.99):
    print(f"\n🔐 ENCRYPTING TEXT: {text[:60]}...")

    bits = to_baudot_bits(text)
    print(f"🧩 Step 1: Baudot Bits (first 80): {bits[:80]}...")

    baudot_bytes = bits_to_bytes(bits)
    print(f"🧩 Step 2: Baudot Bytes (first 20): {baudot_bytes[:20]}...")

    chaos = tent_map(x0, r, len(baudot_bytes))
    print(f"⚡ Step 3: Chaos Map Bytes (first 20): {chaos[:20]}...")

    xored = xor_lists(baudot_bytes, chaos)
    print(f"🔄 Step 4: XOR Result (first 20): {xored[:20]}...")

    aes_enc = aes_encrypt(xored, key)
    print(f"🔒 Step 5: AES Encrypted (base64 first 80): {aes_enc[:80]}...")

    print(f"🧮 Bit Length for Decryption: {len(bits)}")
    return aes_enc, len(bits)

def decrypt(aes_enc, key, bit_len, x0=0.7, r=1.99):
    print("\n🔓 DECRYPTING...")

    xored = aes_decrypt(aes_enc, key)
    print(f"📥 Step 1: AES Decrypted Bytes (first 20): {xored[:20]}...")

    chaos = tent_map(x0, r, len(xored))
    print(f"⚡ Step 2: Chaos Map Regenerated (first 20): {chaos[:20]}...")

    baudot_bytes = xor_lists(xored, chaos)
    print(f"🔄 Step 3: XOR Reversed (first 20): {baudot_bytes[:20]}...")

    bits = bytes_to_bits(baudot_bytes)[:bit_len]
    print(f"🧩 Step 4: Baudot Bits Extracted (first 80): {bits[:80]}...")

    final_text = from_baudot_bits(bits).upper()
    print(f"✅ Step 5: Final Decoded Text (preview): {final_text[:80]}...")
    return final_text

def generate_random_key(length=32):
    return secrets.token_bytes(length)

def main():
    file_path = "text_200kb.txt"
    with open(file_path, 'r') as f:
        raw_text = f.read().lower()

    text = ''.join(c for c in raw_text if c in baudot)
    aes_key = generate_random_key()
    print(f"\n🗝 Random AES Key (Hex): {aes_key.hex()}")

    start_enc = time.time()
    aes_enc, bit_len = encrypt(text, aes_key)
    end_enc = time.time()

    start_dec = time.time()
    decrypted_text = decrypt(aes_enc, aes_key, bit_len)
    end_dec = time.time()

    print(f"\n⏱ Encryption Time: {(end_enc - start_enc) * 1000:.2f} ms")
    print(f"⏱ Decryption Time: {(end_dec - start_dec) * 1000:.2f} ms")

if __name__ == "__main__":
    main()



🗝 Random AES Key (Hex): aab11f308552d3e93f22922030f36aa9201b0d9d81c318d9632b64a3968b1b73

🔐 ENCRYPTING TEXT: vowpv vupqebq del spifyxvr xo brbdmrz oswo yhssbj mozhfs mhe...
🧩 Step 1: Baudot Bits (first 80): 01111000111100101101011110000001111111000110111101100001001111101000001001010000...
🧩 Step 2: Baudot Bytes (first 20): [120, 242, 215, 129, 252, 111, 97, 62, 130, 80, 72, 40, 214, 90, 183, 122, 129, 113, 130, 106]...
⚡ Step 3: Chaos Map Bytes (first 20): [152, 205, 100, 200, 109, 218, 74, 147, 215, 79, 158, 193, 123, 246, 18, 36, 73, 146, 218, 74]...
🔄 Step 4: XOR Result (first 20): [224, 63, 179, 73, 145, 181, 43, 173, 85, 31, 214, 233, 173, 172, 165, 94, 200, 227, 88, 32]...
🔒 Step 5: AES Encrypted (base64 first 80): 0jjbpUNcgHd0eYvjGcqwnRR7fpQSrSf3UrOg8emXWvMKRq3wVDcf9taT6BIFbCc+ifdlrYIS6/vY9B4O...
🧮 Bit Length for Decryption: 1024000

🔓 DECRYPTING...
📥 Step 1: AES Decrypted Bytes (first 20): [224, 63, 179, 73, 145, 181, 43, 173, 85, 31, 214, 233, 173, 172, 165, 94, 200, 227, 88

In [6]:
import time
import secrets
import math
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad, unpad

# === Baudot code (a-z and space only) ===
baudot = {
    ' ': '00000', 't': '00001', 'o': '00011', 'h': '00101', 'n': '00110',
    'm': '00111', 'l': '01001', 'r': '01010', 'g': '01011', 'i': '01100',
    'p': '01101', 'c': '01110', 'v': '01111', 'e': '10000', 'z': '10001',
    'd': '10010', 'b': '10011', 's': '10100', 'y': '10101', 'f': '10110',
    'x': '10111', 'a': '11000', 'w': '11001', 'j': '11010', 'u': '11100',
    'q': '11101', 'k': '11110'
}
reverse_baudot = {v: k for k, v in baudot.items()}

def to_baudot_bits(text):
    return ''.join([baudot[c] for c in text.lower() if c in baudot])

def bits_to_bytes(bits):
    while len(bits) % 8 != 0:
        bits += '0'
    return [int(bits[i:i+8], 2) for i in range(0, len(bits), 8)]

def bytes_to_bits(byte_list):
    return ''.join([format(b, '08b') for b in byte_list])

def from_baudot_bits(bits):
    return ''.join([reverse_baudot.get(bits[i:i+5], '?') for i in range(0, len(bits), 5)])

def cosine_polynomial_map(x0, delta, length):
    x = x0
    chaos = []
    for _ in range(length):
        x = math.cos(delta * x**3 + x)
        chaos.append(int((x + 1) / 2 * 256) % 256)
    return chaos

def xor_lists(a, b):
    return [x ^ y for x, y in zip(a, b)]

# === AES ENCRYPTION ===
def aes_encrypt(byte_list, key):
    key = SHA256.new(key).digest()  # 32 bytes for AES-256
    iv = secrets.token_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(bytes(byte_list), AES.block_size))
    return b64encode(iv + ciphertext).decode()

def aes_decrypt(data_b64, key):
    key = SHA256.new(key).digest()
    raw = b64decode(data_b64)
    iv, ciphertext = raw[:16], raw[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return list(decrypted)
# ======================

def encrypt(text, key, x0=0.7, delta=1.2):
    print(f"\n🔐 ENCRYPTING TEXT: {text[:50]}...")
    bits = to_baudot_bits(text)
    print(f"📡 Baudot Bits: {bits[:80]}... [Total: {len(bits)} bits]")

    baudot_bytes = bits_to_bytes(bits)
    print(f"🧱 Baudot Bytes: {baudot_bytes[:10]}... [Total: {len(baudot_bytes)} bytes]")

    chaos = cosine_polynomial_map(x0, delta, len(baudot_bytes))
    print(f"🎲 Chaotic Values: {chaos[:10]}...")

    xored = xor_lists(baudot_bytes, chaos)
    print(f"🪄 XOR Result: {xored[:10]}...")

    aes_enc = aes_encrypt(xored, key)
    print(f"🔒 AES Encrypted (Base64): {aes_enc[:80]}...")

    print(f"🧮 Baudot Bit Length (for decryption): {len(bits)}")
    return aes_enc, len(bits)

def decrypt(enc_data, key, bit_len, x0=0.7, delta=1.2):
    print("\n🔓 DECRYPTING...")
    xored = aes_decrypt(enc_data, key)
    print(f"📥 Decrypted XOR Bytes: {xored[:10]}...")

    chaos = cosine_polynomial_map(x0, delta, len(xored))
    print(f"🎲 Regenerated Chaotic Values: {chaos[:10]}...")

    baudot_bytes = xor_lists(xored, chaos)
    print(f"🧱 Recovered Baudot Bytes: {baudot_bytes[:10]}...")

    bits = bytes_to_bits(baudot_bytes)[:bit_len]
    print(f"📡 Baudot Bits (Trimmed): {bits[:80]}... [Total: {bit_len} bits]")

    text = from_baudot_bits(bits).upper()
    print(f"✅ Decrypted Text Preview: {text[:80]}...")
    return text

def generate_random_key(length=32):
    return secrets.token_bytes(length)

def main():
    text = "this is a secure message"
    text = ''.join(c for c in text if c in baudot)
    aes_key = generate_random_key()

    print(f"\n🗝️ AES KEY (hex): {aes_key.hex()}")

    start_enc = time.time()
    enc_data, bit_len = encrypt(text, aes_key)
    end_enc = time.time()

    start_dec = time.time()
    dec_text = decrypt(enc_data, aes_key, bit_len)
    end_dec = time.time()

    print(f"\n🔐 Encrypted Data (base64): {enc_data[:80]}...")
    print(f"🔓 Decrypted Text: {dec_text}")
    print(f"⏱️ Encryption Time: {(end_enc - start_enc)*1000:.2f} ms")
    print(f"⏱️ Decryption Time: {(end_dec - start_dec)*1000:.2f} ms")

if __name__ == "__main__":
    main()



🗝️ AES KEY (hex): 4a606bcb890b5346bf438414f8c174b074162e5a44ecab291bcbedad3dfe3696

🔐 ENCRYPTING TEXT: this is a secure message...
📡 Baudot Bits: 00001001010110010100000000110010100000001100000000101001000001110111000101010000... [Total: 120 bits]
🧱 Baudot Bytes: [9, 89, 64, 50, 128, 192, 41, 7, 113, 80]... [Total: 15 bytes]
🎲 Chaotic Values: [184, 237, 124, 255, 52, 214, 192, 229, 150, 253]...
🪄 XOR Result: [177, 180, 60, 205, 180, 22, 233, 226, 231, 173]...
🔒 AES Encrypted (Base64): WqzE3ZubCA034lnikfvhsixCAmWNJ3K/BilFsz0893U=...
🧮 Baudot Bit Length (for decryption): 120

🔓 DECRYPTING...
📥 Decrypted XOR Bytes: [177, 180, 60, 205, 180, 22, 233, 226, 231, 173]...
🎲 Regenerated Chaotic Values: [184, 237, 124, 255, 52, 214, 192, 229, 150, 253]...
🧱 Recovered Baudot Bytes: [9, 89, 64, 50, 128, 192, 41, 7, 113, 80]...
📡 Baudot Bits (Trimmed): 00001001010110010100000000110010100000001100000000101001000001110111000101010000... [Total: 120 bits]
✅ Decrypted Text Preview: THIS IS A SECURE MES

In [16]:
import time
import random
import string
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
import secrets

# Baudot encoding
baudot = {
    'a': '00001', 'b': '00010', 'c': '00011', 'd': '00100', 'e': '00101',
    'f': '00110', 'g': '00111', 'h': '01000', 'i': '01001', 'j': '01010',
    'k': '01011', 'l': '01100', 'm': '01101', 'n': '01110', 'o': '01111',
    'p': '10000', 'q': '10001', 'r': '10010', 's': '10011', 't': '10100',
    'u': '10101', 'v': '10110', 'w': '10111', 'x': '11000', 'y': '11001',
    'z': '11010', ' ': '00000'
}

reverse_baudot = {v: k for k, v in baudot.items()}

def to_baudot_bits(text):
    return ''.join([baudot[c] for c in text.lower() if c in baudot])

def bits_to_bytes(bits):
    while len(bits) % 8 != 0:
        bits += '0'
    return [int(bits[i:i+8], 2) for i in range(0, len(bits), 8)]

def bytes_to_bits(byte_list):
    return ''.join([format(b, '08b') for b in byte_list])

def from_baudot_bits(bits):
    return ''.join([reverse_baudot.get(bits[i:i+5], '?') for i in range(0, len(bits), 5)])

def logistic_map(x0, delta, length):
    x = x0
    chaos = []
    for _ in range(length):
        x = delta * x * (1 - x)
        chaos.append(int(x * 256) % 256)
    return chaos

def xor_lists(a, b):
    return [x ^ y for x, y in zip(a, b)]

def aes_encrypt(byte_list, key_str):
    # Ensure the key is in bytes, if it's a string
    if isinstance(key_str, str):
        key = SHA256.new(key_str.encode()).digest()[:32]  # AES-256
    else:
        key = key_str[:32]  # If key is already in bytes, slice to 32 bytes if necessary

    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(bytes(byte_list))
    encrypted = cipher.nonce + tag + ciphertext
    return b64encode(encrypted).decode()

def aes_decrypt(data_b64, key_str):
    # Ensure the key is in bytes, if it's a string
    if isinstance(key_str, str):
        key = SHA256.new(key_str.encode()).digest()[:32]  # AES-256
    else:
        key = key_str[:32]  # If key is already in bytes, slice to 32 bytes if necessary

    raw = b64decode(data_b64)
    nonce, tag, ciphertext = raw[:16], raw[16:32], raw[32:]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    return list(cipher.decrypt_and_verify(ciphertext, tag))


def encrypt(text, key, x0=0.7, delta=3.9):
    print(f"\n🔐 ENCRYPTING TEXT: {text[:20]}...")
    bits = to_baudot_bits(text)
    print(f"Baudot Bits: {bits}")
    baudot_bytes = bits_to_bytes(bits)
    print(f"Baudot Bytes: {baudot_bytes}")
    chaos = logistic_map(x0, delta, len(baudot_bytes))
    print(f"Logistic Chaotic Bytes: {chaos}")
    xored = xor_lists(baudot_bytes, chaos)
    print(f"XORed Bytes: {xored}")
    aes_enc = aes_encrypt(xored, key)
    print(f"🔒 AES-256 Encrypted (Base64): {aes_enc}")
    print(f"🧮 Baudot Bit Length: {len(bits)} (save this for decryption)")
    return aes_enc, len(bits)

def decrypt(aes_enc, key, bit_len, x0=0.7, delta=3.9):
    print("\n🔓 DECRYPTING...")
    xored = aes_decrypt(aes_enc, key)
    print(f"Decrypted XORed Bytes: {xored}")
    chaos = logistic_map(x0, delta, len(xored))
    print(f"Generated bytes of Logistic Chaotic Bytes: {chaos}")
    baudot_bytes = xor_lists(xored, chaos)
    print(f"Recovered Baudot Bytes: {baudot_bytes}")
    bits = bytes_to_bits(baudot_bytes)[:bit_len]
    print(f"Baudot Bits: {bits}")
    text = from_baudot_bits(bits).upper()
    print(f"✅ Decrypted Plain Text: {text[:50]}...")
    return text

def generate_random_key(length=32):
    # Generate a secure AES-256 key (32 bytes = 256 bits)
    return secrets.token_bytes(length)

def main():
    file_path = "/content/text_200kb.txt"
    with open(file_path, 'r') as f:
        text = f.read().strip()

    aes_key = generate_random_key()  # Generate AES-256 key
    print(f"\n🗝 Random AES Key Used (Hex): {aes_key.hex()}")

    # ENCRYPTION
    start_enc = time.time()
    aes_enc, bit_len = encrypt(text, aes_key)
    end_enc = time.time()

    # DECRYPTION
    start_dec = time.time()
    decrypted_text = decrypt(aes_enc, aes_key, bit_len)
    end_dec = time.time()

    print(f"\n⏱ Encryption Time: {(end_enc - start_enc) * 1000:.2f} ms")
    print(f"⏱ Decryption Time: {(end_dec - start_dec) * 1000:.2f} ms")

if __name__ == "__main__":
    main()


🗝 Random AES Key Used (Hex): b4ff967f437e1c0fc63b0f024261c3aec7642b349e5a30cc38d4c9b463131814

🔐 ENCRYPTING TEXT: vowpv vupqebq del sp...
Baudot Bits: ...
Baudot Bytes: [179, 239, 11, 2, 213, 132, 74, 40, 128, 133, 96, 39, 4, 155, 56, 180, 129, 135, 128, 82, 17, 27, 45, 1, 243, 187, 193, 148, 78, 98, 80, 26, 253, 32, 211, 3, 80, 93, 80, 148, 4, 88, 243, 101, 179, 168, 12, 243, 52, 45, 144, 192, 39, 150, 150, 24, 18, 70, 0, 113, 26, 41, 16, 105, 160, 76, 148, 12, 82, 134, 187, 116, 6, 94, 49, 8, 65, 114, 77, 243, 72, 11, 51, 173, 250, 101, 193, 98, 174, 200, 3, 73, 8, 142, 111, 5, 140, 3, 65, 98, 80, 34, 123, 87, 68, 3, 88, 54, 225, 25, 8, 49, 83, 2, 202, 109, 166, 224, 89, 166, 211, 141, 171, 130, 207, 213, 205, 39, 1, 113, 189, 232, 13, 6, 15, 93, 1, 37, 132, 22, 195, 175, 91, 128, 228, 24, 31, 52, 190, 77, 6, 5, 67, 14, 85, 2, 88, 177, 2, 168, 204, 30, 9, 166, 207, 3, 170, 248, 2, 136, 192, 3, 137, 89, 209, 192, 34, 167, 17, 32, 176, 135, 22, 174, 79, 1, 92, 237, 64, 160, 152, 72, 1

In [1]:
pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.4 MB/s eta 0:00:00
